In [ ]:
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Splitter
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace, BRepBuilderAPI_MakePolygon
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
from OCC.Core.gp import gp_Pnt
from OCC.Core import TopAbs

from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Core.BRep import BRep_Polygon3D
from OCC.Core.TopExp import TopExp_Explorer

from OCC.Core.BOPAlgo import BOPAlgo_Splitter

# Define Polygon and perform split operation

In [ ]:
# See https://dev.opencascade.org/doc/overview/html/occt_user_guides__modeling_algos.html

W = BRepBuilderAPI_MakePolygon()
W.Add(gp_Pnt(-2, -2, 1))
W.Add(gp_Pnt(-2, 2, 1))
W.Add(gp_Pnt(2, 2, 1))
W.Add(gp_Pnt(2, -2, 1))

W.Close()

# At this point, the polygon shape is a "wire frame"
shape1_wire = W.Shape()
print("Shape type of Polygon after .Close()", shape1_wire.ShapeType)
# in order to perform a split with the enture surface of the polygon, we have to convert the wire frame to a face
shape1_face = BRepBuilderAPI_MakeFace(shape1_wire)
print("Shape type of Face", shape1_face.Shape().ShapeType)


box = BRepPrimAPI_MakeBox(gp_Pnt(-0.25, 0, 0), gp_Pnt(0.25, 1, 2)).Shape()
# section_shp = BRepAlgoAPI_Section(shape1_face.Shape(), wall)

splitter = BOPAlgo_Splitter()
splitter.SetNonDestructive(False)
splitter.AddArgument(box)  # arugment means object to cut
splitter.AddTool(shape1_face.Shape())  # tool means arguments are cut by this
splitter.Perform()

result = splitter.Shape()


rnd = JupyterRenderer()
rnd.DisplayShape(result, render_edges=True)

rnd.Display()

# Problem: the result is still a compound structure of the upper and lower half-box
- This can be seen by inspecting the ``result.ShapeType`` property. It should read ``TopoDS_Compound``

In [ ]:
print(result.ShapeType)

# Decompose the split results
* There are probably multiple ways to decompose the results of the split operation, but one was is to use the `TopExp_Explorer`.

In [ ]:
exp = TopExp_Explorer()
exp.Init(result, TopAbs.TopAbs_SOLID)

sub_shapes = []
while exp.More():
    sub_shapes.append(exp.Current())
    exp.Next()

In [ ]:
rnd = JupyterRenderer()
colors = ["#DB0570", "#0506DB"]
for s, c in zip(sub_shapes, colors):
    rnd.DisplayShape(s, render_edges=True, shape_color=c)

rnd.Display()

# Educational example: what if we used the wire frame shape rather than the face to perform the split.
* We define the split frame at the center of the box surfaces
* This does not cut the solid into two pieces, but rather cuts the surfaces into pieces.
* As result, the explorer only returns a single solid.
* The shape type of the result is `Solid` rather than `Compound`

In [ ]:
# See https://dev.opencascade.org/doc/overview/html/occt_user_guides__modeling_algos.html

W = BRepBuilderAPI_MakePolygon()
W.Add(gp_Pnt(-0.25, -0.25, 1))
W.Add(gp_Pnt(0.25, -0.25, 1))
W.Add(gp_Pnt(0.25, 0.25, 1))
W.Add(gp_Pnt(-0.25, 0.25, 1))

W.Close()

# At this point, the polygon shape is a "wire frame"
shape1_wire = W.Shape()
print("Shape type of Polygon after .Close()", shape1_wire.ShapeType)


box = BRepPrimAPI_MakeBox(gp_Pnt(-0.25, -0.25, 0), gp_Pnt(0.25, 0.25, 2)).Shape()
# section_shp = BRepAlgoAPI_Section(shape1_face.Shape(), wall)

splitter = BOPAlgo_Splitter()
splitter.SetNonDestructive(False)
splitter.AddArgument(box)  # arugment means object to cut
splitter.AddTool(shape1_wire)  # tool means arguments are cut by this
splitter.Perform()

result = splitter.Shape()
print("Shape type of result", result.ShapeType)


exp = TopExp_Explorer()
exp.Init(result, TopAbs.TopAbs_SOLID)

sub_shapes = []
while exp.More():
    sub_shapes.append(exp.Current())
    exp.Next()

print("Sub shapes after split", sub_shapes)


rnd = JupyterRenderer()
# rnd.DisplayShape(result, render_edges=True)
rnd.DisplayShape(result, render_edges=True)

rnd.Display()